In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras as tfk

from sklearn.preprocessing import StandardScaler
from keras.callbacks import Callback

from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, LayerNormalization, RepeatVector

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
plt.style.use('seaborn')

from tensorflow.keras.preprocessing import timeseries_dataset_from_array

from utils import preprocess_time_series_v2

Using TensorFlow backend.


In [2]:
df = pd.read_csv('REE_2016_2020_gluonTS.csv')


In [3]:
df = pd.read_csv('REE_2016_2020_gluonTS.csv')

X_train, y_train, X_val, y_val, X_test, y_test, scaler = preprocess_time_series_v2(df, date_init='2016', date_final='2017', date_val='2018', date_test='2019', 
                            past_timestamps=96, forecasting_horizon=6, 
                            standarize=True, metafeatures=False, tendency=False)

                      Date  value
0      2016-01-02 23:00:00  24151
1      2016-01-03 00:00:00  22170
2      2016-01-03 01:00:00  20691
3      2016-01-03 02:00:00  19737
4      2016-01-03 03:00:00  19437
...                    ...    ...
34940  2019-12-28 19:00:00  29282
34941  2019-12-28 20:00:00  29266
34942  2019-12-28 21:00:00  28022
34943  2019-12-28 22:00:00  25964
34944  2019-12-28 23:00:00  24680

[34945 rows x 2 columns]
(17497, 1)
(8760, 1)
(8688, 1)
shape train/val (17497, 1) (8760, 1) (8688, 1)
Tamano de la serie temporal 2
Tamano de la serie temporal 2
Tamano de la serie temporal 2
shape train/val (17396, 96, 1, 1) (8659, 96, 1, 1) (8587, 96, 1, 1)
(17396, 96, 1, 1)
/home/blasting/TFG_TimeSeries/notebooks/utils.py:133: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  val_data = df[date_val].to_numpy()
/home/bl

In [4]:
input_train = tfk.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
encoder_last_h1, encoder_last_h2, encoder_last_c = LSTM(
 80, dropout=0.1, recurrent_dropout=0.1, 
 return_sequences=False, return_state=True)(input_train)
encoder_last_h1 = tfk.layers.BatchNormalization(momentum=0.6)(encoder_last_h1)
encoder_last_c = tfk.layers.BatchNormalization(momentum=0.6)(encoder_last_c)

decoder = RepeatVector(y_train.shape[1])(encoder_last_h1)
decoder = LSTM(80, dropout=0.1, recurrent_dropout=0.1, return_state=False, return_sequences=True)(
    decoder, initial_state=[encoder_last_h1, encoder_last_c])
print(decoder)

outputs = TimeDistributed(Dense(1))(decoder)

model = tfk.Model(inputs=input_train, outputs=outputs)
model.compile(loss=tfk.losses.MeanSquaredError(), optimizer= tfk.optimizers.Adam(),
metrics=[tfk.metrics.MeanAbsoluteError()])
model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 6, 80), dtype=tf.float32, name=None), name='lstm_1/transpose_1:0', description="created by layer 'lstm_1'")
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 1)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 80), (None,  26240       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 80)           320         lstm[0][0]                       
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 

In [11]:
batch_size = 256
history = model.fit(X_train, y_train, 
                    batch_size=batch_size, 
                    epochs=10, 
                    validation_data=(X_val, y_val)) 


loss = history.history['loss']            #entrenamiento
val_loss = history.history['val_loss']    #validacion
#val_mse = history.history['val_mse']    #validacion
#mse = history.history['mse']    #validacion


Epoch 1/10
68/68 [==============================] - 26s 387ms/step - loss: 0.0316 - mean_absolute_error: 0.1255 - val_loss: 0.0256 - val_mean_absolute_error: 0.1062
Epoch 2/10
68/68 [==============================] - 23s 334ms/step - loss: 0.0330 - mean_absolute_error: 0.1286 - val_loss: 0.0277 - val_mean_absolute_error: 0.1143
Epoch 3/10
68/68 [==============================] - 23s 332ms/step - loss: 0.0327 - mean_absolute_error: 0.1278 - val_loss: 0.0229 - val_mean_absolute_error: 0.1022
Epoch 4/10
68/68 [==============================] - 20s 296ms/step - loss: 0.0313 - mean_absolute_error: 0.1262 - val_loss: 0.0267 - val_mean_absolute_error: 0.1085
Epoch 5/10
68/68 [==============================] - 22s 317ms/step - loss: 0.0323 - mean_absolute_error: 0.1278 - val_loss: 0.0260 - val_mean_absolute_error: 0.1101
Epoch 6/10
68/68 [==============================] - 21s 313ms/step - loss: 0.0314 - mean_absolute_error: 0.1264 - val_loss: 0.0239 - val_mean_absolute_error: 0.1016
Epoch 7/10

In [12]:
def inverse_transform(yhat, y, scaler):
	return scaler.inverse_transform(y),  scaler.inverse_transform(yhat)

In [13]:
y_hat = model.predict(X_test)
y_hat_2 = model.predict(X_val)

y_test2, y_hat = inverse_transform(y_hat, y_test, scaler)
y_val2, y_hat_2 = inverse_transform(y_hat_2, y_val, scaler)

In [14]:
from sklearn.metrics import mean_absolute_error
def evaluate_forecasts(test, forecasts, n_seq):
	for i in range(n_seq):
		actual = [row[i] for row in test]
		predicted = [forecast[i] for forecast in forecasts]
		rmse = mean_absolute_error(actual, predicted)
		print('t+%d MSE: %f' % ((i+1), rmse))

In [15]:
print("test")
evaluate_forecasts(y_test2, y_hat, 6)

print("validacion")
evaluate_forecasts(y_val2, y_hat_2, 6)

test
t+1 MSE: 339.420176
t+2 MSE: 412.268886
t+3 MSE: 474.754075
t+4 MSE: 528.673938
t+5 MSE: 566.340296
t+6 MSE: 593.365637
validacion
t+1 MSE: 318.964585
t+2 MSE: 392.016248
t+3 MSE: 454.373082
t+4 MSE: 507.370392
t+5 MSE: 540.634629
t+6 MSE: 556.963158


In [16]:
(318.964585 + 392.016248 + 454.373082 + 507.370392 + 540.634629 + 556.963158)/6

461.720349